In [4]:
import pandas as pd
import numpy as np

# Ensure datetime conversion
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')

# 1. Ride Length in minutes
df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
df = df[df['ride_length'] > 0]  # remove negative or zero durations

# 2. Ride Date-related columns
df['ride_date'] = df['started_at'].dt.date
df['ride_month'] = df['started_at'].dt.month_name()
df['ride_year'] = df['started_at'].dt.year
df['day_of_week'] = df['started_at'].dt.day_name()
df['ride_start_time'] = df['started_at'].dt.time
df['ride_end_time'] = df['ended_at'].dt.time

# 3. Ride Distance (Haversine Formula)
from math import radians, cos, sin, asin, sqrt

def haversine(row):
    lat1, lon1, lat2, lon2 = row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']
    if pd.isnull(lat1) or pd.isnull(lat2) or pd.isnull(lon1) or pd.isnull(lon2):
        return np.nan
    
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of earth in kilometers
    return c * r

df['ride_distance_km'] = df.apply(haversine, axis=1)

# 4. Season (based on ride month)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'

df['season'] = df['started_at'].dt.month.map(get_season)


In [5]:
df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,ride_length,ride_date,ride_month,ride_year,day_of_week,ride_start_time,ride_end_time,ride_distance_km,season
0,4422E707103AA4FF,electric_bike,2024-10-14 03:26:00,2024-10-14 03:32:00,NaN,NaN,NaN,NaN,41.96,-87.65,...,member,6.0,2024-10-14,October,2024,Monday,03:26:00,03:32:00,2.771218,Autumn
1,19DB722B44CBE82F,electric_bike,2024-10-13 19:33:00,2024-10-13 19:39:00,NaN,NaN,NaN,NaN,41.98,-87.67,...,member,6.0,2024-10-13,October,2024,Sunday,19:33:00,19:39:00,1.385570,Autumn
2,20AE2509FD68C939,electric_bike,2024-10-13 23:40:00,2024-10-13 23:48:00,NaN,NaN,NaN,NaN,41.97,-87.66,...,member,8.0,2024-10-13,October,2024,Sunday,23:40:00,23:48:00,2.372640,Autumn
3,D0F17580AB9515A9,electric_bike,2024-10-14 02:13:00,2024-10-14 02:25:00,NaN,NaN,NaN,NaN,41.95,-87.65,...,member,12.0,2024-10-14,October,2024,Monday,02:13:00,02:25:00,1.111949,Autumn
4,A114A483941288D1,electric_bike,2024-10-13 19:26:00,2024-10-13 19:28:00,NaN,NaN,NaN,NaN,41.98,-87.67,...,member,2.0,2024-10-13,October,2024,Sunday,19:26:00,19:28:00,0.000000,Autumn


In [6]:
raw_clean_path = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/Trip_Full_Year_Cleaned/clean_raw_2024_data.csv"
df.to_csv(raw_clean_path, index=False)


In [1]:
import os
import pandas as pd

# Folder where your raw data files are stored
folder_path = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/raw_data"
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]

df_list = []

original_columns = [
    'ride_id', 'rideable_type', 'started_at', 'ended_at',
    'start_station_name', 'start_station_id', 'end_station_name',
    'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
    'member_casual'
]

for file in csv_files:
    temp_df = pd.read_csv(file, low_memory=False)
    
    # Keep only original columns
    temp_df = temp_df[[col for col in original_columns if col in temp_df.columns]]
    
    df_list.append(temp_df)

# Merge all clean, raw data
df = pd.concat(df_list, ignore_index=True)


In [ ]:
import os
import pandas as pd

# Path to raw data files
folder_path = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/raw_data"
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]

df_list = []

for file in csv_files:
    temp_df = pd.read_csv(file, low_memory=False)

    # Fix the column name if it exists under a different alias
    if 'ride_length_distance_km' in temp_df.columns:
        temp_df.rename(columns={'ride_length_distance_km': 'ride_distance_km'}, inplace=True)

    # Check if the standardized column exists before proceeding
    if 'ride_distance_km' in temp_df.columns:
        df_list.append(temp_df)
    else:
        print(f"Missing column in file (skipped): {file}")

# Final merged DataFrame
df = pd.concat(df_list, ignore_index=True)


In [ ]:
# Convert columns to datetime
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
df['ride_date'] = pd.to_datetime(df['ride_date'], errors='coerce')
df['ride_start_time'] = pd.to_datetime(df['ride_start_time'], errors='coerce')
df['ride_end_time'] = pd.to_datetime(df['ride_end_time'], errors='coerce')

# Convert ride_distance_km to float
df['ride_distance_km'] = pd.to_numeric(df['ride_distance_km'], errors='coerce')


In [ ]:
# Define function
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

# Create numeric month if missing
df['month_num'] = pd.to_datetime(df['ride_date'], errors='coerce').dt.month

# Create season column
df['season'] = df['month_num'].apply(get_season)


In [ ]:
# View summary info
df.info()

# Check for any missing values
df.isnull().sum()


In [ ]:
output_path = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/Trip_Full_Year_Cleaned/cleaned_2024_cyclistic_data.csv"
df.to_csv(output_path, index=False)


In [ ]:
# STEP 1: Load all CSVs without forcing dtype
df_list = [pd.read_csv(file, low_memory=False) for file in csv_files]

# STEP 2: Combine all months into one DataFrame
df = pd.concat(df_list, ignore_index=True)

# STEP 3: Clean the problematic column (e.g., 'ride_distance_km')
# Replace known bad strings with NaN, then convert to float
df['ride_distance_km'] = pd.to_numeric(df['ride_distance_km'], errors='coerce')


In [ ]:
import pandas as pd

# Load just one file to inspect columns
sample = pd.read_csv(csv_files[0])
print(sample.columns)


In [ ]:
# Convert columns to appropriate types
df['ride_id'] = df['ride_id'].astype(str)
df['rideable_type'] = df['rideable_type'].astype('category')
df['start_station_name'] = df['start_station_name'].astype('category')
df['end_station_name'] = df['end_station_name'].astype('category')
df['member_casual'] = df['member_casual'].astype('category')
df['ride_Year'] = df['ride_Year'].astype('int')
df['ride_Month'] = pd.Categorical(df['ride_Month'],
    categories=['January', 'February', 'March', 'April', 'May', 'June',
                'July', 'August', 'September', 'October', 'November', 'December'],
    ordered=True
)
df['Day_of_week'] = pd.Categorical(df['Day_of_week'],
    categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    ordered=True
)


In [ ]:
# Map months to seasons
season_map = {
    'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
    'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
    'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
    'September': 'Fall', 'October': 'Fall', 'November': 'Fall'
}

df['season'] = df['ride_Month'].map(season_map)


In [ ]:
import pandas as pd
import numpy as np
import os
import glob

# Folder path
folder_path = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/raw_data"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

for file in csv_files:
    try:
        temp_df = pd.read_csv(file, low_memory=False)

        # Fix inconsistent column names
        if 'ride_length_distance_km' in temp_df.columns:
            temp_df.rename(columns={'ride_length_distance_km': 'ride_distance_km'}, inplace=True)

        # Ensure datetime fields are parsed correctly
        temp_df['started_at'] = pd.to_datetime(temp_df['started_at'], errors='coerce')
        temp_df['ended_at'] = pd.to_datetime(temp_df['ended_at'], errors='coerce')

        # Generate new time-related fields
        temp_df['ride_Length'] = (temp_df['ended_at'] - temp_df['started_at']).dt.total_seconds() / 60
        temp_df['ride_Date'] = temp_df['started_at'].dt.date
        temp_df['ride_Month'] = temp_df['started_at'].dt.month_name()
        temp_df['ride_Year'] = temp_df['started_at'].dt.year
        temp_df['Day_of_week'] = temp_df['started_at'].dt.day_name()
        temp_df['ride_start_time'] = temp_df['started_at'].dt.time
        temp_df['ride_end_time'] = temp_df['ended_at'].dt.time

        # Clean distance column
        if 'ride_distance_km' in temp_df.columns:
            temp_df['ride_distance_km'] = pd.to_numeric(temp_df['ride_distance_km'], errors='coerce')
        else:
            temp_df['ride_distance_km'] = np.nan  # Ensure consistency

        df_list.append(temp_df)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all cleaned data
df = pd.concat(df_list, ignore_index=True)
print("✅ Merge complete. Combined shape:", df.shape)


In [ ]:
for file in csv_files:
    temp_df = pd.read_csv(file, low_memory=False)
    if 'ride_length_distance_km' not in temp_df.columns:
        print(f"Missing column in file: {file}")


In [ ]:
if 'ride_length_distance_km' in temp_df.columns:
    temp_df['ride_length_distance_km'] = pd.to_numeric(temp_df['ride_length_distance_km'], errors='coerce')


In [ ]:
import pandas as pd
import os 


# Path to your raw data folder
data_folder = "/Users/winifred/Downloads/Cyclistic_Bike_Share_Project/raw_data"

# List of all 12 CSV files
csv_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith(".csv")]

# Load and combine
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)
